In [37]:
! pip install faiss-cpu readabilipy

In [38]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [39]:
from dotenv import load_dotenv
import os

current_directory = os.getcwd()
print(current_directory)

# take environment variables from .env.
load_dotenv(dotenv_path="../../../.env")


/workspace/resume-coach/backend/packages/import-job/import_job


True

In [42]:
from langchain_community.document_loaders import WebBaseLoader
from import_job.text_transformers import ReadibilityPyDocumentTransformer

loader = WebBaseLoader("https://www.linkedin.com/jobs/view/3860575851")
transformer = ReadibilityPyDocumentTransformer()

docs = loader.load()
docs_transformed = transformer.transform_documents(docs)

pretty_print_docs(docs_transformed)

Document 1:

Google hiring Software Engineering Manager II, Machine Learning, Google Cloud in San Francisco, CA | LinkedIn Agree & Join LinkedIn By clicking Continue, you agree to LinkedIn’s User Agreement, Privacy Policy, and Cookie Policy. Skip to main content LinkedIn Software Engineering Manager II, Machine Learning, Google Cloud in San Jose, CA Expand search This button displays the currently selected search type. When expanded it provides a list of search options that will switch the search inputs to match the current selection. Jobs People Learning Clear text Clear text Clear text Clear text Clear text Join now Sign in Software Engineering Manager II, Machine Learning, Google Cloud Google San Francisco, CA Apply Join or sign in to find your next job Join to apply for the Software Engineering Manager II, Machine Learning, Google Cloud role at Google Not you? Remove photo First name Last name Email Password (6+ characters) By clicking Agree & Join, you agree to the LinkedIn User A

In [43]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs_transformed)

vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import ChatPromptTemplate
from langchain_openai import OpenAI

model = OpenAI(temperature=0)

prompt = ChatPromptTemplate.from_template("""You are givent plain text contents of a web page of a job posting. Contents may include text from navigational elements like headers, footers, sidebars etc. Your task is to questions related to the job description. :

<job_posting_page>
{context}
</job_posting_page>

Question: {input}""")

document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [45]:
question = 'who is the employer?'

docs = retriever.get_relevant_documents(question)

# pretty_print_docs(docs)

# out = retrieval_chain.invoke(question)
# print("out:", out)


In [25]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(model)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    question
)
pretty_print_docs(compressed_docs)

/usr/local/python/3.11.8/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/python/3.11.8/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/python/3.11.8/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/python/3.11.8/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

Emeryville, CA 5 months ago Engineering Manager, API Engineering Manager, API Asana San Francisco, CA $220,000.00 - $336,000.00 2 weeks ago Software Engineering Manager Software Engineering Manager U.S. Bank San Francisco, CA $135,150.00 - $174,900.00 2 weeks ago Director of Engineering Director of Engineering Siena AI San Francisco, CA 1 week ago Software Development Director Software Development Director Oracle Redwood City, CA $120,300.00 - $291,900.00 1 week ago
----------------------------------------------------------------------------------------------------
Document 2:

Google hiring Software Engineering Manager II, Machine Learning, Google Cloud in San Francisco, CA | LinkedIn Agree & Join LinkedIn By clicking Continue, you agree to LinkedIn’s User Agreement, Privacy Policy, and Cookie Policy. Skip to main content LinkedIn Software Engineering Manager II, Machine Learning, Google Cloud in San Jose, CA Expand search This button displays the currently selected searc